In [1]:
import os
#os.listdir('/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/')
os.listdir('/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/')
#os.listdir('/home/em-student/experiment_boom/ctb_cov_eq_50_1/leaks/sim_input/')


['id_360996.si',
 'id_17313.cov',
 'id_0.si',
 'id_360996.cov',
 'id_17313.si',
 'id_606718.si',
 'id_0.cov',
 'id_606718.cov']

In [2]:
# input files
in_file = '/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/id_606718.si'
#in_file = '/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/id_8536.si'
#in_file = '/home/em-student/experiment_boom/ctb_cov_eq_50_1/leaks/sim_input/id_574.si'


In [3]:
# config
import tempfile
from src.utils import *
from Config import ROCKET_CONF, BOOM_CONF
outdir = tempfile.mkdtemp()


(mutator, preprocessor, hscHost) = setupHSC('Template', outdir, proc_num=0, debug=True, contract='ct', isa='RV64IM')

(sim_input, (data_a, data_b), assert_intr) = mutator.read_siminput(in_file)

(hsc_input, rtl_input, symbols) = preprocessor.process(sim_input, data_a, data_b, False)

toplevel, bin_dir, v_file, cov_len = ROCKET_CONF

In [4]:
# process and run
res_file = 'results_{}.xml'.format(0)

debug = 1
cmd = 'make SIM_BUILD={} VFILE={} TOPLEVEL={} DEBUG={} INPUT={} COCOTB_RESULTS_FILE={}'.format(bin_dir, v_file, toplevel, debug, rtl_input, res_file)
p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print(p.stderr.decode())

sys:1: ResourceWarning: unclosed file <_io.TextIOWrapper name=2 mode='w' encoding='UTF-8'>



In [5]:
print(p.stdout.decode())

make results_0.xml
make[1]: Entering directory '/home/em-student/tools/hw-sw-contract-fuzzer/Fuzzer'
MODULE=HWSim TESTCASE= TOPLEVEL=RocketTile TOPLEVEL_LANG=verilog \
        /home/em-student/experiment_4/fuzz_bin/RocketTile +DEBUG=1 
     -.--ns INFO     cocotb.gpi                         ..mbed/gpi_embed.cpp:100  in set_program_name_in_venv        Using Python virtual environment interpreter at /home/em-student/tools/hw-sw-contract-fuzzer/.direnv/python-3.8.10/bin/python
     -.--ns INFO     cocotb.gpi                         ../gpi/GpiCommon.cpp:105  in gpi_print_registered_impl       VPI registered
     -.--ns INFO     cocotb.gpi                         ..mbed/gpi_embed.cpp:240  in _embed_sim_init                 Python interpreter initialized and cocotb loaded!
     0.00ns INFO     cocotb                                      __init__.py:220  in _initialise_testbench_          Running on Verilator version 4.106 2020-12-02
     0.00ns INFO     cocotb                                

In [6]:
leak = any(filter(lambda x: '[Leakage]' in str(x), p.stdout.splitlines()))
# if debug == 1:
#     temp = b''
#     for l in filter(lambda x: "idx" in str(x),p.stdout.splitlines()):
#         if l != temp:
#             print(l)
#             temp = l
dir, fname = os.path.split(rtl_input)
fname = fname.split('.si')[0]
cov_name = fname + '.cov'
cov_out =  os.path.join(dir, cov_name)

# fd = open(os.path.join(dir, fname + '.symbols'), 'rb')
# ct = fd.read()
# print(shake_128(ct).hexdigest(8))
# fd = open(os.path.join(dir, fname + '.hex'), 'rb')
# ct = fd.read()
# print(shake_128(ct).hexdigest(8))

b = bitarray()
fd = open(cov_out, 'rb')
b.fromfile(fd)
fd.close()


In [7]:
fb = open(os.path.join(outdir, '.input_0_a.S'))
fb_lines = fb.readlines()
for l in fb_lines:
    print(l)

# Assembly file template for fuzzing



#*********************************************************************

# test_template.S

#---------------------------------------------------------------------



#include "riscv_test.h"

#include "test_macros.h"

#include "fuzz_test.h"



#define TESTEND gp



RVTEST_RV64UF



// RVTEST_CODE_BEGIN

        .section .text.init;

        .align  6;

        .weak stvec_handler;

        .weak mtvec_handler;

        .global _start;

_start:

        /* reset vector */

INIT_XREGS

        j reset_vector;

        .align;

init_freg:

INIT_FREGS

        ret

        .align;

trap_stvec:

        clear_sie;

_fuzz_suffix:

_s0:;

        /* Fuzz trap */

        /**/

_end_suffix:

        ecall

trap_mtvec: 

       clear_mie;

  DUMP_REGS

        

  write_tohost:

        li TESTEND, 1

        sw TESTEND, tohost, t5;

  _test_end:

        j write_tohost;

reset_vector:

        csrwi mstatus, 0;

        RISCV_MULTICORE_DISABLE;

        DE

In [8]:
os.listdir(outdir)

['.input_0_a.elf',
 '.input_0.hex',
 '.input_0.symbols',
 '.input_0_a.S',
 '.input_0_b.elf',
 '.input_0.cov',
 '.input_0_b.S',
 '.input_0.si']